In [6]:
import pandas as pd
import json
#Converts a dataframe to a geojson Point output
def df_to_geojson_point(df, properties, lat, lon):
    geojson = {'type':'FeatureCollection', 'features':[]}
    for _, row in df.iterrows():
        feature = {'type':'Feature',
                   'properties':{},
                   'geometry':{'type':'Point',
                               'coordinates':[]}}
        feature['geometry']['coordinates'] = [row[lon],row[lat]]
        for prop in properties:
            feature['properties'][prop] = row[prop]
        geojson['features'].append(feature)
    return geojson

#Converts a dataframe to a geojson LineString output
def df_to_geojson_line(df, ycoord, xcoord, aggregateBy):
    
    #create new dataframe, aggregate points per bear to a linestring:
    uniqueValues=[]
    for _, row in df.iterrows():
        uniqueValues.append(row[aggregateBy])
    
    
    uniqueValues=list(set(uniqueValues))    
    
    aggDFlist=[]
    for uniqueValue in uniqueValues:
        coords=[]
        for _, row in df.iterrows():
            if row[aggregateBy] == uniqueValue:
                currx = row[xcoord]
                curry = row[ycoord]
                #coords+=",["+str(currx)+","+str(curry)+"]"
                coords.append([row[xcoord],row[ycoord]])
        
        #coords="["+coords[1:]+"]"
        aggDFlist.append([uniqueValue,coords])
    
    
    
    aggr_df = pd.DataFrame(aggDFlist, columns=["bear_name", "coords"])  
    
    properties=["bear_name"]
    geojson = {"type":"FeatureCollection", "features":[]}
    for _, row in aggr_df.iterrows():
        feature = {"type":"Feature",
                   "properties":{},
                   "geometry":{"type":"LineString",
                               "coordinates": ""}}
        #print row["coords"]
        feature["geometry"]["coordinates"] = row["coords"]
        for prop in properties:
            feature["properties"][prop] = row[prop]
        geojson["features"].append(feature)
    return str(geojson).replace("'",'"')



In [7]:
from IPython.display import Javascript
#Create a javascript variable with our geojson data to visualize in the browser
#The data object 'vizObj' will be a global varialbe in our window that
#We can pass to another javascript function call

#-------------------------------------------------------------------------------
csvfile=r'./Data/Brown bear Slovenia 1993-1999UTM.csv'
df = pd.read_csv(csvfile, low_memory=False)

properties=[]
properties.append('timestamp')
properties.append('time_index')
properties.append('individual-local-identifier')

#create time index
def createTimeIndex (row):
   index = row['timestamp'][:4]+row['timestamp'][5:7]+row['timestamp'][8:10]
   return index
df['time_index'] = df.apply(lambda row: createTimeIndex (row),axis=1)
#print df.head(10)

#get subset for bear
#df_subset = df[df['individual-local-identifier'] == 'dusan'] #[:20] #test, just first 20 points
df_subset = df

#get unique bear names:
#bears = set(list(df['individual-local-identifier']))
#print bears

#EDIT HERE PTS
geojson_points = df_to_geojson_point(df_subset, properties, 'location-lat', 'location-long')
#print geojson_points

#EDIT HERE LINES
geojson_lines = df_to_geojson_line(df_subset, 'location-lat', 'location-long', 'individual-local-identifier')
#print geojson_lines

Javascript("""window.vizObj_lines1={};""".format(geojson_lines))
Javascript("""window.vizObj_points1={};""".format(geojson_points))






<IPython.core.display.Javascript object>

In [8]:
#Now let's make some HTML to style our intended mapbox output
from IPython.display import HTML
HTML("""
<link href='https://www.mapbox.com/base/latest/base.css' rel='stylesheet' />
<style> #map {
  position: relative;
  width: auto;
  height: 400px;
  overflow:visible;
}

.legend {
    background-color: #fff;
    border-radius: 3px;
    bottom: 30px;
    box-shadow: 0 1px 2px rgba(0,0,0,0.10);
    font: 9px/10px 'Helvetica Neue', Arial, Helvetica, sans-serif;
    padding: 5px;
    position: absolute;
    right: 5px;
    z-index: 1;
}
.legend h4 {
    margin: 0 0 10px;
}
.legend div span {
    border-radius: 50%;
    display: inline-block;
    height: 10px;
    margin-right: 5px;
    width: 10px;
}
</style>

""")

In [9]:
%%javascript
require.config({
  paths: {
      mapboxgl: 'https://api.tiles.mapbox.com/mapbox-gl-js/v0.21.0/mapbox-gl',
      bootstrap: 'https://maxcdn.bootstrapcdn.com/bootstrap/3.3.6/js/bootstrap.min'
  }
});

<IPython.core.display.Javascript object>

In [10]:
%%javascript
IPython.OutputArea.auto_scroll_threshold = 9999;
require(['mapboxgl', 'bootstrap'], function(mapboxgl, bootstrap){
    mapboxgl.accessToken = 'pk.eyJ1Ijoiam9oYW5uZXN1aGwiLCJhIjoiY2l1dWJ4YndlMDJ5ZzJvbjBhNHAybnhxYSJ9.Tvsyxnah2xGhl9kfJnt8Yg';
    var map = new mapboxgl.Map({
        container: 'map', // container id
        style: 'mapbox://styles/mapbox/dark-v8', //stylesheet location
        center: [14.3031661581, 45.8698704914], // starting position
        zoom: 6 // starting zoom
    });

    function addSegLayerPoint(mapid) {
        //alert(window.vizObj_points.features[0].geometry.coordinates);
        try {  
            map.addSource("points", {
                "type": "geojson",
                "data": window.vizObj_points1
            });
            map.addLayer({
                "id": "points",
                "type": "symbol",
                "source": "points",
                "type": "circle",
                "paint": {
                    "circle-radius": 4,
                    "circle-color":{
                        property: 'individual-local-identifier',
                        type: 'categorical',
                        stops: [
                            ['mishko','#a6cee3'],
                            ['ancka','#1f78b4'],
                            ['milan','#b2df8a'],
                            ['clio','#33a02c'],
                            ['klemen','#fb9a99'],
                            ['vanja','#e31a1c'],
                            ['srecko','#fdbf6f'],
                            ['jana','#ff7f00'],
                            ['polona','#cab2d6'],
                            ['urosh','#6a3d9a'],
                            ['lucia','#ffff99'],
                            ['vera','#b15928'],
                            ['ivan','#7fc97f'],
                            ['nejc','#beaed4'],
                            ['jure','#fdc086'],
                            ['maja','#ffff99'],
                            ['dusan','#386cb0'],
                            ['dinko','#e41a1c'],
                            ['metka','#377eb8'],
                            ['vinko','#4daf4a'],
                            ['joze','#984ea3'],
                            ['janko','#ff7f00']]  
                    }
                } 
                
            }); 
            
        } catch (err) {
            console.log(err);
            alert(err);
        }
    };
    
    function addSegLayerLine(mapid) {
        // Mapbox GL JS Api - import segment
        //alert(window.vizObj_lines.features[0].geometry.coordinates[0]);
        
        var segment_src = new mapboxgl.GeoJSONSource({
            data: window.vizObj_lines1,  
            maxzoom: 18,
            buffer: 1,
            tolerance: 1
        });
        try {
            mapid.addSource('segment', segment_src);
            mapid.addLayer({
                id: 'segment',
                type: 'line',
                source: 'segment',
                paint: {
                    "line-opacity": 1,
                    "line-width": 0.5,
                    "line-color": '#ffffff'                        
                    }   
                }
            );
        } catch (err) {
            alert(err);
            console.log(err);
        }
    };

    map.once('style.load', function(e) {        
        addSegLayerLine(map);
        addSegLayerPoint(map);
        map.addControl(L.mapbox.legendControl()); //not working?
        //map.addControl(new mapboxgl.Navigation({  //not working?
           //position: 'top-left'
        //}));
        map.addControl(new mapboxgl.Navigation({position: 'top-left'}));
    });
    
    

    // Create a popup, but don't add it to the map yet.
    var popup = new mapboxgl.Popup({
        closeButton: false,
        closeOnClick: false
    });
    map.on('mousemove', function(e) {
        var features = map.queryRenderedFeatures(e.point, { layers: ['points'] });
        // Change the cursor style as a UI indicator.
        map.getCanvas().style.cursor = (features.length) ? 'pointer' : '';

        if (!features.length) {
            popup.remove();
            return;
        }
        var feature = features[0];
        // Populate the popup and set its coordinates
        // based on the feature found.
        popup.setLngLat(feature.geometry.coordinates)
            .setHTML(feature.properties.timestamp)
            .addTo(map);
    });     
    
    
});
element.append('<div id="map"></div>');
element.append("<div id='legend' class='legend'> \
    <h4>Trajectories of individual bears</h4> \
    <div><span style='background-color: #1f78b4'></span>ancka </div> \
    <div><span style='background-color: #33a02c'></span>clio  </div> \
    <div><span style='background-color: #e41a1c'></span>dinko </div> \
    <div><span style='background-color: #386cb0'></span>dusan </div> \
    <div><span style='background-color: #7fc97f'></span>ivan  </div> \
    <div><span style='background-color: #ff7f00'></span>jana  </div> \
    <div><span style='background-color: #ff7f00'></span>janko </div> \
    <div><span style='background-color: #984ea3'></span>joze  </div> \
    <div><span style='background-color: #fdc086'></span>jure  </div> \
    <div><span style='background-color: #fb9a99'></span>klemen</div> \
    <div><span style='background-color: #ffff99'></span>lucia </div> \
    <div><span style='background-color: #ffff99'></span>maja  </div> \
    <div><span style='background-color: #377eb8'></span>metka </div> \
    <div><span style='background-color: #b2df8a'></span>milan </div> \
    <div><span style='background-color: #a6cee3'></span>mishko</div> \
    <div><span style='background-color: #beaed4'></span>nejc  </div> \
    <div><span style='background-color: #cab2d6'></span>polona</div> \
    <div><span style='background-color: #fdbf6f'></span>srecko</div> \
    <div><span style='background-color: #6a3d9a'></span>urosh </div> \
    <div><span style='background-color: #e31a1c'></span>vanja </div> \
    <div><span style='background-color: #b15928'></span>vera  </div> \
    <div><span style='background-color: #4daf4a'></span>vinko </div> \
    </div>");
               




<IPython.core.display.Javascript object>